In [7]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

In [8]:
# gpu 사용
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [50]:
# 텐서 변환
transform = transforms.Compose([
    transforms.ToTensor()
])

# 데이터셋
train_data = ImageFolder(
    root = './custom_data/train_data', 
    transform = transform
)

test_data = ImageFolder(
    root = './custom_data/test_data',
    transform = transform
)

In [51]:
# 데이터로더
train_data_loader = DataLoader(
    dataset = train_data, 
    batch_size = 8, 
    shuffle = True, 
    num_workers = 2
)

test_data_loader = DataLoader(
    dataset = test_data,
    batch_size = 8,
    shuffle = True,
    num_workers = 2
)

In [36]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, 5), # 입력채널, 출력채널, 필터사이즈
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.layer3 = nn.Sequential(
            nn.Linear(16 * 13 * 29, 120),
            nn.ReLU(),
            nn.Linear(120, 2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        #print(out.shape)
        out = out.view(out.size(0), -1)
        #print(out.shape)
        out = self.layer3(out)
        #print(out.shape)
        
        return out

In [37]:
model = CNN().to(device)

In [38]:
#values = torch.Tensor(3, 3, 64, 128).to(device)
#result = model(values)

In [42]:
optimizer = optim.Adam(model.parameters(), lr = 1e-5)
criterion = nn.CrossEntropyLoss().to(device)

In [44]:
total_batch = len(train_data_loader)

nb_epochs = 5
for epoch in range(nb_epochs):
    avg_loss = 0
    
    for num, samples in enumerate(train_data_loader):
        x_train, y_train = samples
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(x_train)
        loss = criterion(hypothesis, y_train)
        loss.backward()
        optimizer.step()
        
        avg_loss += loss / total_batch
        
    print("Epoch: {} | Loss: {}".format(epoch + 1, avg_loss))

Epoch: 1 | Loss: 0.2122827172279358
Epoch: 2 | Loss: 0.16381892561912537
Epoch: 3 | Loss: 0.12599624693393707
Epoch: 4 | Loss: 0.09646501392126083
Epoch: 5 | Loss: 0.07402625679969788


In [46]:
torch.save(model.state_dict(), './model/model.pth')

In [47]:
new_model = CNN().to(device)

In [48]:
new_model.load_state_dict(torch.load('./model/model.pth'))

<All keys matched successfully>

In [49]:
print(model.layer1[0])
print(new_model.layer1[0])

print(model.layer1[0].weight[0][0][0]) # convolusion layer 출력
print(new_model.layer1[0].weight[0][0][0])

model.layer1[0].weight[0] == new_model.layer1[0].weight[0] # 저장 전 후 모델 비교

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.1049,  0.0905,  0.1144,  0.0290,  0.0592], device='cuda:0',
       grad_fn=<SelectBackward>)
tensor([-0.1049,  0.0905,  0.1144,  0.0290,  0.0592], device='cuda:0',
       grad_fn=<SelectBackward>)


tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]], device='cuda:0')

In [52]:
with torch.no_grad():
    for num, samples in enumerate(test_data_loader):
        x_test, y_test = samples
        x_test.to(device)
        y_test.to(device)
        
        prediction = new_model(x_train)
        correct_prediction = torch.argmax(prediction, 1) == y_train
        accuracy = correct_prediction.float().mean()
        
        print("Accuracy: {}".format(accuracy.item()))

Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
